In [ ]:
def strategy_7(data):
    # Calculate Ichimoku Cloud components
    def calculate_tenkan_sen(high, low):
        return (high.rolling(window=9).max() + low.rolling(window=9).min()) / 2

    def calculate_kijun_sen(high, low):
        return (high.rolling(window=26).max() + low.rolling(window=26).min()) / 2

    def calculate_senkou_span_a(tenkan_sen, kijun_sen):
        return ((tenkan_sen + kijun_sen) / 2).shift(26)

    def calculate_senkou_span_b(high, low):
        return ((high.rolling(window=52).max() + low.rolling(window=52).min()) / 2).shift(26)

    def calculate_chikou_span(close):
        return close.shift(-26)

    # Copy data for output
    output = data.copy()

    # Calculate Ichimoku components
    data['conv_line'] = calculate_tenkan_sen(data['high'], data['low'])
    data['base_line'] = calculate_kijun_sen(data['high'], data['low'])
    data['leadS_a'] = calculate_senkou_span_a(data['conv_line'], data['base_line'])
    data['leadS_b'] = calculate_senkou_span_b(data['high'], data['low'])
    data['lagS'] = calculate_chikou_span(data['close'])

    # Calculate ADX
    def adx(data):
        adx_period = 12  # Period for ADX
        data['ADX'] = ta.ADX(data['high'], data['low'], data['close'], timeperiod=adx_period)
        return data

    data = adx(data)

    # Calculate MACD and Signal Line
    data['MACD'] = data['high'].ewm(span=12, adjust=False).mean() - data['high'].ewm(span=26, adjust=False).mean()
    data['signal_line'] = data['MACD'].rolling(window=9).mean()

    # Calculate Internal Bar Strength (IBS) and weighted average
    data['IBS'] = (data['close'] - data['low']) / (data['high'] - data['low'])
    data['weight_avg'] = data['IBS'].ewm(span=5, adjust=False).mean()

    # Calculate Chaikin Money Flow (CMF)
    n = 10
    data['MF_Multiplier'] = (2 * data['close'] - data['low'] - data['high']) / (data['high'] - data['low'])
    data['MF_Volume'] = data['MF_Multiplier'] * data['volume']
    data['CMF'] = data['MF_Volume'].rolling(n).sum() / data['volume'].rolling(n).sum()
    data = data.drop(['MF_Multiplier', 'MF_Volume'], axis=1)

    # Initialize signals
    data['signals'] = 0

    # Generate trading signals
    for i in range(len(data)):
        if data['conv_line'][i] > data['base_line'][i] and data['close'][i] > data['leadS_a'][i] and data['close'][i] > data['leadS_b'][i] and data['MACD'][i] > data['signal_line'][i] and data['CMF'][i] > 0:
            data['signals'][i] = 1  # Buy signal
        elif data['conv_line'][i] < data['base_line'][i] and data['close'][i] < data['leadS_a'][i] and data['close'][i] < data['leadS_b'][i] and data['MACD'][i] < data['signal_line'][i] and data['CMF'][i] < 0:
            data['signals'][i] = -1  # Sell signal

            
    return data
